# Introduction to NLP fundamentals in Tensorflow
NLP has the goal of driving information out of natural language(could be sequence text or speech).

Another common term for NLP problems is sequence to sequence problems.

In [1]:
!nvidia-smi 

Thu Dec  1 18:56:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1. ## import helper functions

In [2]:
#Download them to colab and import the functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-12-01 18:56:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-12-01 18:56:34 (95.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,compare_historys,unzip_data,walk_through_dir

#Get a text dataset.
 The dataset we're going to be using is kaggle's introduction to NLP dataset(text sample of Tweets labelled as disaster or not disaster).

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-12-01 18:56:37--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 74.125.24.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-12-01 18:56:37 (116 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
#Unzip the data
unzip_data('nlp_getting_started.zip')

#2. Become one with the data.
Visualize,Visualize,visualize

In [6]:
#Visualizing a text data

# To visualize our data we have to read them first with python.
#Another way is to use pandas library.

In [7]:
import pandas as pd
#Pandas have a read_csv function
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()
 


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [9]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [10]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
#How many examples of each class?
#It tells us if it's balanced data or not
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
# What  are the total no. of samples
#The total no. of ID (individual samples)
len(train_df),len(test_df)

(7613, 3263)

In [14]:
#Let's visualize some random samples
import random
random_index=random.randint(0,len(train_df)-5)# Create random indexes not hgher than the total on. of samples
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
# getting the random indexes of the text and target dataset:
   _,text,target=row # we have added the '-' because the index is also returned and we don't want that
   print(f'Target: {target}','(real_disaster)' if target>0 else '(not a real disaster)')
   print(f'Text:\n{text}\n')
   print('--\n')

Target: 0 (not a real disaster)
Text:
samel_samel has a crush: http://t.co/tBsTk5VqU0

--

Target: 1 (real_disaster)
Text:
Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into water I called my daughters and said t...

--

Target: 1 (real_disaster)
Text:
The Drought Is Real ??????

--

Target: 1 (real_disaster)
Text:
#3Novices : Renison mine sees seismic event http://t.co/2i4EOGGO5j A small earthquake at Tasmania's Renison tin project has created a temÛ_

--

Target: 1 (real_disaster)
Text:
The Rocky Fire burning in northern California is still threatening more than 7000 buildings. It's the largest in the state #9newsmornings

--



#3. Creating a validation dataset

##Split data into training and validatioin sets

In [15]:
from sklearn.model_selection import train_test_split
#use it to split the train data into trtain and val data
train_sentence,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                      train_df_shuffled['target'].to_numpy(),
                                                                      test_size=0.1,
                                                                      random_state=42)

In [16]:
#Check the lengths
len(train_sentence),len(train_labels),len(val_sentences),len(val_labels)


(6851, 6851, 762, 762)

In [17]:
#Check them
train_sentence[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

#4. Convert text to numbers

###Tokenization Vs Embedding

Tokenization has a disadvantage where every word or character is converted to one hot encoding.

Example: Let's have 10,000 different words then to represent those we use a 10,000 long one hot encoding but 

If we use Embedding, we can represent it using a vector of any dimension we like for s single word or charcter.

**ALSO NOTE** that embedding is like a weight matrix and becomes a layer in our network and can be learned.

Text Vectorization (tokenization)

In [18]:
train_sentence[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [20]:
#use the defaut text vectorizer parameters
text_vectorizer=TextVectorization(max_tokens=None,#how many words in the vocabulary(automatically add <oov>)
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace', 
                                  ngrams=None, #create groups of n-words
                                  output_mode='int',
                                  output_sequence_length=None,
                                  pad_to_max_tokens=False)

In [21]:
#Find the average number of tokens(words) in the training
round(sum([len(i.split()) for i in train_sentence])/len(train_sentence))

15

In [22]:
#Setup text vectorization variables
max_vocab_length=10000
max_length=15 #Average length

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                output_mode='int',
                output_sequence_length=max_length)

In [23]:
# Map text vectorizer ot adapt it to the training data


#5. Fit the text vectorizer to the training data
 we can use the instance of TextVectorization which is the 'textvectorizer'.

**USING THE ADAPT METHOD**

In [24]:
text_vectorizer.adapt(train_sentence)

In [25]:
#Create a sample sentence and tokenize it
sample_sentence='There is  a flood in my street'
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [26]:
#choose a random sentence from the training dataset and tokenize it
random_sentence=random.choice(train_sentence)
print(f'original txt:\n{random_sentence}\n\n vectorized version')
text_vectorizer([random_sentence])

original txt:
Aircraft debris found on island is from MH370 Malaysia confirms http://t.co/X3RccHKagO

 vectorized version


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 661,  247,  225,   11,  419,    9,   20,  171,  242, 1538,    1,
           0,    0,    0,    0]])>

In [27]:
#Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() #get the unique words sorted by commonality ascending
top_5_words=words_in_vocab[:5] # 5 most common words
top_5_words

['', '[UNK]', 'the', 'a', 'in']

In [28]:
least_common_5_words=words_in_vocab[-5:]
least_common_5_words

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

#Creating an Embedding Layer

To make our Embedding, we're going to use the *tensorflow embedding layer*

The parameters we most care about in the embedding layer:

* input_dim = size of our vocabulary.
* output_dim = size of the output embedding vector.
* input_length= length of the sequences being passed to the embedding layer.

In [29]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, #set input shape
                             output_dim=128, #output_shape
                             embeddings_initializer='uniform',
                             input_length=max_length #how long is each input
                             )



In [30]:
#Get random sentence from training sentence
random_sentence=random.choice(train_sentence)
print(f'Original text:\n{random_sentence}\n\n Embedded version:')
#Embedd the random sentence (turn it into dense vectors of fixed size )
sample_embed=embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Explosion in Gaza Strip kills four wounds 30; cause unknown http://t.co/GopSiCP8bm via @Reuters

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0175607 , -0.01058873, -0.04148157, ..., -0.01385028,
          0.01898583, -0.04933142],
        [-0.03839401,  0.02220476, -0.02383045, ...,  0.00717516,
          0.00655509, -0.01870319],
        [-0.04925835,  0.02340833,  0.04765958, ...,  0.01621986,
          0.04186392,  0.02533469],
        ...,
        [ 0.02672959,  0.02261036,  0.02129758, ...,  0.01338193,
          0.00024067,  0.03131247],
        [ 0.04838011, -0.04359727,  0.02343065, ...,  0.01744376,
         -0.01663082, -0.02401304],
        [ 0.04838011, -0.04359727,  0.02343065, ...,  0.01744376,
         -0.01663082, -0.02401304]]], dtype=float32)>

In [31]:
#check out a single token's embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-1.75606981e-02, -1.05887279e-02, -4.14815657e-02, -4.82442863e-02,
        -1.76717043e-02, -4.87318896e-02,  9.72043350e-03, -4.94457781e-04,
        -2.51912717e-02, -2.34937072e-02, -4.30444852e-02, -2.47663148e-02,
        -7.26289675e-03,  3.16922329e-02, -4.47200797e-02, -3.18786055e-02,
         2.41508335e-03, -2.97256950e-02, -1.29342079e-05,  6.61606714e-03,
        -3.88069041e-02, -3.62743512e-02,  4.40895669e-02,  3.74270715e-02,
         2.54295804e-02, -4.19213399e-02, -1.95829272e-02,  2.07787752e-03,
        -2.82208454e-02,  1.74671151e-02, -2.88150311e-02, -3.12006474e-03,
         1.47769116e-02,  3.76959890e-03, -2.60616671e-02, -1.69798359e-02,
         3.61374505e-02,  1.17517710e-02,  2.22494937e-02,  1.23360269e-02,
        -1.43917315e-02,  4.94885705e-02, -4.18570638e-02, -1.81538686e-02,
        -3.37655097e-02,  4.03301790e-03,  4.95316051e-02, -3.29379439e-02,
        -4.31870483e-02, -4.33830731e-0

# Modelling a text dataset(running a series of experiments)
 
 NOw we have got ways to turn our sequences into numbers,it's time to start building a series of modelling experiments.

 WE'll start with a baseline and move on from there.

 * Model 0: Naive'Bayes model(**NON DEEP LEARNING**)

 importing it from scikit learn

 * Model 1: Feed forward neural network(dense _model)
 * model 2: LSTM_model(rnn)
 * model 3: GRU_model(rnn)
 * model 4: Bidirectional_LSTM model(rnn)
 * model 5: 1D convolutional neural network(CNN)
 * model 6: Tensorflow HUb(using feature extractionfor NLP)
 * model 7: Same as model 6 but only with 10 % of the data.

###MODEL _0
As with all ML experiments it's needed to create a base_line model so as to evaluate the others on this benchmark.

We will use the SciKit learn method(NAive Bayes which is not a Deep Learning model)


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
 # we are also going to build a pipe line
from sklearn.pipeline import Pipeline
#Create tokenization and modelling pipeline
model_0=Pipeline([
    ('tfidf',TfidfVectorizer()), # convert words to number tfidf
    ('clf',MultinomialNB()) # model the text #clf is short for classifier
])
#Fit the model on the train_sentences and train _labels
model_0.fit(train_sentence,train_labels) # Don't need an epoch

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [33]:
#Then we evaluate the base_line model
baseline_score=model_0.score(val_sentences,val_labels) #Score means as evaluate is to tensorflow
print(f"Our base_model achieves an accuracy of :{baseline_score*100:.2f}%")

Our base_model achieves an accuracy of :79.27%


In [34]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

**That is not bad for a balanced data...so it's  better than guessing**

In [35]:
#Make prediction
baseline_preds= model_0.predict(val_sentences)
baseline_preds[:20]


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

*Let's use metrics for classification other than accuracy like precision(false postives),recall(false negatives),f1_score(combination of both)...*

In [36]:
#Define a function to compute this metric to compute predictions deviation from true label.

import sklearn
import tensorflow as tf
def metrics_computing(val_labels,predictions):
  accuracy=tf.keras.metrics.Accuracy(val_labels,predictions)
  precision=tf.keras.metrics.Precision(val_labels,predictions)
  recall=tf.keras.metrics.Recall(val_labels,predictions)
  f1_score= sklearn.metrics.f1_score(val_labels,predictions)
  return accuracy,precision,recall,f1_score
 

*Or we can easily use the accuracy_score and precision_recall_fscore_support*

In [37]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true,y_pred,average='weighted')
  #Then put them in dictionary
  model_results={'accuracy':model_accuracy,
                 'precision':model_precision,
                 'recall':model_recall,
                 'f1':model_f1}
  return model_results


In [38]:
#Then we are going to check the function
baseline_results=calculate_results(val_labels,baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

##Then we create the second model which is  a DNN 

In [39]:
#Create a tensorboard_callback
from helper_functions import create_tensorboard_callback
#create a directory to save Tensorboard logs
SAVE_DIR='model_logs'

In [40]:
#Build the model with FUnctional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype=tf.string) #inputs are one dimensional strings
x=text_vectorizer(inputs)
x=embedding(x) #create an embedding vectors
x=layers.GlobalAveragePooling1D()(x) #condense the feature vector for each token 
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs)

In [41]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-train

In [42]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [43]:
outputs.shape


TensorShape([None, 1])

In [44]:
train_labels[0].shape

()

In [45]:
model_1.history=model_1.fit(train_sentence,train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                  experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20221201-185642
Epoch 1/5
215/215 [==============================] - 4s 5ms/step - loss: 0.6109 - accuracy: 0.6951 - val_loss: 0.5351 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4415 - accuracy: 0.8168 - val_loss: 0.4707 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3467 - accuracy: 0.8596 - val_loss: 0.4573 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2836 - accuracy: 0.8910 - val_loss: 0.4650 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2369 - accuracy: 0.9123 - val_loss: 0.4791 - val_accuracy: 0.7782


In [46]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4791 - accuracy: 0.7782


[0.47908705472946167, 0.778215229511261]

In [47]:
model_1_pred=model_1.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [48]:
model_1_pred[:10],val_labels[:10]

(array([[0.38108402],
        [0.8428245 ],
        [0.9980172 ],
        [0.13964595],
        [0.11416081],
        [0.9472031 ],
        [0.9269345 ],
        [0.99466914],
        [0.97369856],
        [0.30848643]], dtype=float32), array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0]))

We need to compare our prediction with the val_labels...so we need to round and squeeze because our preds has extra dimensiona and also decimal.

In [49]:
#first get rid if te 1 dimension using tf.
#And then round it that takes >0.5 to 1 and <0.5 to 0
model_1_pred=tf.squeeze(tf.round(model_1_pred))

In [50]:
model_1_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [51]:
#Then we can use the metrics function to compare
model_1_results=calculate_results(y_true=val_labels,y_pred=model_1_pred)
model_1_results

{'accuracy': 77.82152230971128,
 'precision': 0.7807522349051432,
 'recall': 0.7782152230971129,
 'f1': 0.7758810170952618}

In [52]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

We then can now compare to baseline_results with the model_1_results

This is the advantage of using the dic data structure

In [53]:
import numpy as np
np.array(list(model_1_results.values()))> np.array(list(baseline_results.values()))

array([False, False, False, False])

So our base model is better than the DNN we built

So let's try another model that can beat the baseline_model....But before that let's visualize and understand embeddings.

## Visualizing learned embeddings

In [54]:
#Get the vocabulary from teh text vecttorizer layer
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [55]:
#Model 1 summary()
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-train

In [56]:
#Get teh weight matrix of embedding layer
#(these are the numerical representation of each token in our training data)
embed_weights=model_1.get_layer('embedding').get_weights()[0]

In [57]:
embed_weights.shape

(10000, 128)

# We can visualize using word2vec projector

In [58]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [59]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Recurrent Neural Network(RNN'S)
RNN's are useful for sequence data.
The premise of a recurrent neural network is to use the representation of a previous to aid the representation of a later input.

##MOdel 2:LSTM
LSTM= long short term memory(one of the most popular RNN layers).

Our structure of an RNN typically looks like this:
Input > tokenizer > Embedding > Layers(RNNs/Dense) > outputs label(probability)

In [60]:
#Create an LSTM model
from tensorflow.keras import layers
inputs=tf.keras.layers.Input(shape=(1,),dtype='string')
x= text_vectorizer(inputs)
x= embedding(x)
x=layers.LSTM(64,return_sequences=True)(x) #WHen your stacking RNNS you need to set return sequence = True
x= layers.LSTM(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(inputs,outputs,name='model_2_LSTM')

In [61]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

The default activation for LSTM layers and ther RNNS are tanh functions

In [62]:
#let's compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [63]:
model_2.fit(train_sentence,
            train_labels,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20221201-185657
Epoch 1/5
215/215 [==============================] - 10s 11ms/step - loss: 0.2167 - accuracy: 0.9240 - val_loss: 0.6207 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1581 - accuracy: 0.9421 - val_loss: 0.6293 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1330 - accuracy: 0.9492 - val_loss: 0.7219 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1110 - accuracy: 0.9578 - val_loss: 0.8941 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0884 - accuracy: 0.9654 - val_loss: 1.0190 - val_accuracy: 0.7730


###Make predictions with LSTM model

In [64]:
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[1.8546239e-02],
       [6.9137448e-01],
       [9.9995577e-01],
       [1.8836288e-02],
       [2.0772280e-04],
       [9.9969137e-01],
       [9.8704231e-01],
       [9.9996364e-01],
       [9.9993658e-01],
       [4.1387990e-01]], dtype=float32)

In [65]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [66]:
#Then we will squeeze and round the predictions to compare to val_labels
model_2_pred_probs=tf.squeeze(tf.round(model_2_pred_probs))

In [67]:
model_2_pred_probs

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0

In [68]:
#then we will call the calculater results
model_2_results=calculate_results(y_true=val_labels,y_pred=model_2_pred_probs)

In [69]:
model_2_results

{'accuracy': 77.29658792650919,
 'precision': 0.7740517401498704,
 'recall': 0.7729658792650919,
 'f1': 0.7712160418848196}

In [70]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [71]:
#Still our baseline_model results seem better than the model_2

In [72]:
#Let's again build another layer to enhance

### We will then build the GRU layer(gated recurrent unit) model_3

In [73]:
inputs=tf.keras.layers.Input(shape=(1,),dtype=tf.string)

Let's build a GRU powered RNN

In [74]:
x= text_vectorizer(inputs)
x= embedding(x)
x= tf.keras.layers.GRU(64)(x)
#x= tf.keras.layers.LSTM(64,return_sequences=True)(x)
#x=tf.keras.layers.GRU(64)(x)
#x= tf.keras.layers.Dense(64,activation='relu')(x)
outputs= tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_3=tf.keras.Model(inputs,outputs,name='Model_3_GRU')


In [75]:
model_3.summary()

Model: "Model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [76]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history=model_3.fit(train_sentence,train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_3_gru')])


Saving TensorBoard log files to: model_logs/model_3_gru/20221201-185715
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.1654 - accuracy: 0.9348 - val_loss: 0.6910 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0842 - accuracy: 0.9675 - val_loss: 0.7526 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0711 - accuracy: 0.9721 - val_loss: 0.8841 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0588 - accuracy: 0.9742 - val_loss: 1.1792 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0490 - accuracy: 0.9781 - val_loss: 1.1767 - val_accuracy: 0.7756


In [77]:
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs

24/24 [==============================] - 0s 3ms/step


array([[1.88773638e-03],
       [8.28554094e-01],
       [9.99920607e-01],
       [6.27682731e-02],
       [7.95629167e-05],
       [9.99747813e-01],
       [9.37772632e-01],
       [9.99950409e-01],
       [9.99906778e-01],
       [6.65291965e-01],
       [2.06846453e-04],
       [9.73193526e-01],
       [3.17830709e-04],
       [5.14669195e-02],
       [9.98684918e-05],
       [7.68722442e-04],
       [3.10529984e-04],
       [2.78604653e-04],
       [2.35527363e-02],
       [9.99779522e-01],
       [9.86316144e-01],
       [5.60956105e-05],
       [9.99118507e-01],
       [3.46652023e-03],
       [9.99924183e-01],
       [9.99938726e-01],
       [3.75150034e-04],
       [5.35737898e-04],
       [2.52133119e-04],
       [3.23794603e-01],
       [9.92580891e-01],
       [8.96783313e-04],
       [6.76400168e-03],
       [1.81501056e-03],
       [4.07583266e-01],
       [1.13571770e-01],
       [9.99861002e-01],
       [2.68809229e-01],
       [6.73282845e-03],
       [9.99946237e-01],


In [78]:
#WE squeeze and round them to compare to val_labels
model_3_pred_probs=tf.squeeze(tf.round(model_3_pred_probs))

In [79]:
#Then we can get the results to compare to the baseline_results
model_3_results=calculate_results(y_true=val_labels,y_pred=model_3_pred_probs)

In [80]:
model_3_results

{'accuracy': 77.55905511811024,
 'precision': 0.7794121823201254,
 'recall': 0.7755905511811023,
 'f1': 0.7726614113025507}

In [81]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Still not better when it comes to accuracy,so we will proceed to model_4(bi_directional)

In [82]:
inputs=tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(x)
model_4=tf.keras.Model(inputs,outputs,name='model_4_bidirectional')




In [83]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                             

In [84]:
#Then we will compile and fit our model
model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4.fit(train_sentence,train_labels,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_4_bidriectional')])

Saving TensorBoard log files to: model_logs/model_4_bidriectional/20221201-185729
Epoch 1/5
215/215 [==============================] - 9s 18ms/step - loss: 0.0968 - accuracy: 0.9645 - val_loss: 1.2129 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0490 - accuracy: 0.9785 - val_loss: 1.3209 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0506 - accuracy: 0.9777 - val_loss: 1.3103 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0465 - accuracy: 0.9785 - val_loss: 1.4297 - val_accuracy: 0.7375
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0395 - accuracy: 0.9800 - val_loss: 1.6907 - val_accuracy: 0.7638


In [85]:
#make predoiction and evaluat it
model_4_pred_probs=model_4.predict(val_sentences)

24/24 [==============================] - 1s 5ms/step


In [86]:
model_4_pred_probs=tf.squeeze(tf.round(model_4_pred_probs))

In [87]:
model_4_results=calculate_results(y_true=val_labels,y_pred=model_4_pred_probs)

In [88]:
model_4_results,baseline_results

({'accuracy': 76.37795275590551,
  'precision': 0.7649838845988884,
  'recall': 0.7637795275590551,
  'f1': 0.761768588939328},
 {'accuracy': 79.26509186351706,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706,
  'f1': 0.7862189758049549})

Still it performs less than our baseline_model, so we will build our 5th model using conv1D layers...so far we have tried SimpleRNN,LSTM,GRU,bidirectional LSTM...Now we are going to try the conv1D layer

## Convolutional Neural Networks for Text(and other types of sequences)

model_5-conv1d

In [89]:
#Test out our embedding layer,conv1D layer and max pooling layer
embedding_test=embedding(text_vectorizer(['this is a test sentence']))
conv_1d=tf.keras.layers.Conv1D(filters=32,kernel_size=3,activation='relu',padding='valid')
conv_1d_output=conv_1d(embedding_test)
max_pool=tf.keras.layers.GlobalMaxPooling1D()
max_pool_output=max_pool(conv_1d_output)

In [90]:
embedding_test.shape,conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 13, 32]), TensorShape([1, 32]))

If we have seen how the output shapes turn out to be...let' then use this and build the model.

In [91]:
inputs=tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=tf.keras.layers.Conv1D(64,5,activation='relu')(x)
x= tf.keras.layers.GlobalAveragePooling1D()(x)
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(x)
model_5=tf.keras.Model(inputs,outputs)


In [92]:
model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_5.fit(train_sentence,train_labels,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20221201-185759
Epoch 1/5
215/215 [==============================] - 3s 6ms/step - loss: 0.1586 - accuracy: 0.9480 - val_loss: 0.8948 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0883 - accuracy: 0.9658 - val_loss: 1.0487 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0687 - accuracy: 0.9723 - val_loss: 1.2222 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0593 - accuracy: 0.9749 - val_loss: 1.3041 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0517 - accuracy: 0.9775 - val_loss: 1.4240 - val_accuracy: 0.7612


In [93]:
model_5_pred_probs=model_5.predict(val_sentences)

24/24 [==============================] - 0s 2ms/step


In [94]:
model_5_pred_probs=tf.squeeze(tf.round(model_5_pred_probs))

In [95]:
model_5_results=calculate_results(y_true=val_labels,y_pred=model_5_pred_probs)

In [120]:
model_5_results

{'accuracy': 75.7217847769029,
 'precision': 0.7570143518442038,
 'recall': 0.7572178477690289,
 'f1': 0.7561520587599432}

Still not beating our baseline results do we will try to use transfer learning in NLP

Model_6 :

will be built using a transfer learning model using tensorflow hub feature extraction.

In [96]:
import tensorflow_hub as hub
embed=hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embedding=embed([sample_sentence])

In [97]:
print(embedding[0][0:50]) 

tf.Tensor(
[-0.01602834  0.01068852  0.02425466 -0.01405768  0.01434426  0.08292625
  0.01963371  0.06160141 -0.00352699 -0.01216414  0.00978647 -0.01248496
  0.01232345  0.09748451  0.06141113 -0.03728351  0.01860887 -0.04669854
  0.00413913 -0.06363908 -0.02469897  0.02713693  0.02284445 -0.00210027
 -0.00630592 -0.03964962  0.02220407  0.00115075 -0.03132175  0.00119526
 -0.04012549  0.04561896 -0.01530597 -0.00175916  0.02173136 -0.08450424
  0.03340026  0.04604554 -0.02480249 -0.08681663  0.00702694 -0.00770479
 -0.01434536  0.07814164 -0.10676058 -0.05152992 -0.00858159 -0.03232232
 -0.03871096  0.0258147 ], shape=(50,), dtype=float32)


In [98]:
#So now we just builf some kind of Dense layer on top of the embedding layer
sentence_encoder_layer=hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                      input_shape=[],
                                      dtype=tf.string,
                                      trainable=False,
                                      name='USE')




In [110]:
#Define the sequential API
model_6=tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid')
],name='model_6_use')

In [111]:
model_6.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6.summary()

Model: "model_6_use"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [112]:
#train a classifier on top of USE pretrained embeddings
model_6_history=model_6.fit(train_sentence,train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'tf_hub_sentence_encoder')])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20221201-193021
Epoch 1/5
215/215 [==============================] - 4s 14ms/step - loss: 0.5047 - accuracy: 0.7884 - val_loss: 0.4489 - val_accuracy: 0.8058
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4139 - accuracy: 0.8170 - val_loss: 0.4381 - val_accuracy: 0.8097
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4011 - accuracy: 0.8225 - val_loss: 0.4377 - val_accuracy: 0.8045
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3931 - accuracy: 0.8282 - val_loss: 0.4345 - val_accuracy: 0.8084
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3868 - accuracy: 0.8279 - val_loss: 0.4289 - val_accuracy: 0.8163


In [103]:
model_6_pred_probs=model_6.predict(val_sentences)

24/24 [==============================] - 1s 17ms/step


In [104]:
model_6_pred_probs=tf.squeeze(tf.round(model_6_pred_probs))

In [105]:
model_6_results=calculate_results(y_true=val_labels,y_pred=model_6_pred_probs)

In [106]:
model_6_results

{'accuracy': 79.00262467191601,
 'precision': 0.7907348439526268,
 'recall': 0.7900262467191601,
 'f1': 0.7887751851796453}

In [107]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}